<a href="https://colab.research.google.com/github/carinadourado/carinadourado/blob/main/insper_07_web_scraping_resumo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Web Scraping

### Capturando a página

- Visite a página que contém o dado que você precisa
- Acesse o código-fonte (HTML) para verificar se a informação consta no HTML original.
  - Se sim:
    - Crie um programa para baixar o HTML (baixe a URL que você visitou inicialmente)
  - Se não:
    - Descubra, através da aba **Network**, qual requisição trouxe os dados (a que trouxe o HTML é a primeira - provavelmente foi alguma do tipo `xhr` ou `fetch`)
    - Ao encontrar a requisição, verifique:
      - URL
      - Método (em geral vai ser `GET` ou `POST`)
        - Caso o método seja `POST`, verifique quais dados foram enviados no formulário (você precisará enviá-los na requisição)
      - Formato dos dados de resposta (HTML, JSON etc.)

> Caso necessite baixar os dados via `POST`, utilize: `resposta = requests.post(url, data=dados)`, onde `dados` é um dicionário (chave-valor) com os dados do formulário que você deverá submeter.

Existem casos em que `requests` não vai ser suficiente:
- Pode ser que em alguns casos você precise executar bibliotecas mais complexas, como `selenium`, para conseguir capturar as informações que necessita.
- Quando você precisa fazer muitas requisições, use a biblioteca `scrapy`.


### Extraindo os dados

- Caso a página baixada seja um HTML, você pode utilizar diversas técnicas para extrair os dados:
  - Manipulação de strings (`.split`, `.strip`, `.lower` etc.)
  - Expressões regulares (`import re`)
  - XPath (`pip install lxml`)
  - Outras bibliotecas de mais alto nível (`pip install beautifulsoup4`)
- Caso a página seja um JSON, você pode utilizar: `dados = resposta.json()`
- Caso esteja em um outro formato, você precisará buscar uma biblioteca que consiga lidar com esse formato.

### Exemplos

- Dados no HTML: [Links de CSVs no OpenDataSUS](https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8)
- Dados requisitados de maneira assíncrona, via JavaScript: [Verbete da Wikipedia](https://pt.wikipedia.org/wiki/Python) (passe o mouse em cima de um link para outro verbete)
- Dados requisitados de maneira assíncrona via `POST`:
  - [De Olho na Fila](https://deolhonafila.prefeitura.sp.gov.br/index.php)
  - [Barragens de Mineração](https://app.anm.gov.br/SIGBM/Publico/GerenciarPublico)

# Voltaremos às 21:10.

In [ ]:
import requests

url = "https://app.anm.gov.br/SIGBM/Publico/GerenciarPublico"
dados = {
  "startIndex": "0",
  "pageSize": "1000",
  "orderProperty": "TotalPontuacao",
  "orderAscending": "false",
  "DTOGerenciarFiltroPublico[CodigoBarragem]": "",
  "DTOGerenciarFiltroPublico[NecessitaPAEBM]": "0",
  "DTOGerenciarFiltroPublico[BarragemInseridaPNSB]": "0",
  "DTOGerenciarFiltroPublico[PossuiBackUpDam]": "0",
  "DTOGerenciarFiltroPublico[SituacaoDeclaracaoCondicaoEstabilidade]": "0",
}
resposta = requests.post(url, data=dados)

In [ ]:
import json
resultado = json.loads(resposta.text)

In [ ]:
barragens = resposta.json()
print(len(barragens["Entities"]))
#for barragem in barragens["Entities"]:
#  print(f"{barragem['Codigo']}: {barragem['NomeBarragem']}")

883


In [ ]:
import pandas as pd
barragens = resposta.json()
df = pd.DataFrame(barragens["Entities"])
df.columns

Index(['Chave', 'CodigoBarragemEncrypted', 'CodigoDeclaracaoAtual', 'Codigo',
       'Posicao', 'TotalPontuacao', 'NomeBarragem', 'Empreendedor',
       'Municipio', 'UF', 'CategoriaRisco', 'DanoPotencialAssociado',
       'Classificacao', 'Descadastramento', 'PontuacaoEstadoConservacao',
       'ExistePendenciasEmAberto', 'ExisteOficioBarragem', 'CpfCnpj',
       'NecessitaPaebm', 'BarragemInseridaPnsb', 'SituacaoNivelEmergencial',
       'SituacaoDeclaracaoCondicaoEstabilidade', 'CodigoVisivel'],
      dtype='object')

In [ ]:
url_declaracao = "https://app.anm.gov.br/SIGBM/BarragemPublico/ListarUsinas?idDeclaracao="
for barragem in barragens["Entities"]:
  url_declaracao_barragem = url_declaracao + str(barragem["CodigoDeclaracaoAtual"])
  resposta_usinas = requests.get(url_declaracao_barragem)
  print(resposta_usinas.json())

{'Lista': '[{"Codigo":305,"NomeUsina":"PLANTA ENGENHO D\\u0027ÁGUA","CpfCnpj":7950015000160}]'}
{'Lista': '[{"Codigo":305,"NomeUsina":"PLANTA ENGENHO D\\u0027ÁGUA","CpfCnpj":7950015000160}]'}
{'Lista': '[]'}
{'Lista': '[]'}
{'Lista': '[{"Codigo":51,"NomeUsina":"FÁBRICA (ITM UMD FAB, ITM SEC FAB, ITM PNIV FAB, ITM SEC M1 e ITM SEC M2)","CpfCnpj":33592510004494}]'}
{'Lista': '[]'}
{'Lista': '[]'}
{'Lista': '[{"Codigo":51,"NomeUsina":"FÁBRICA (ITM UMD FAB, ITM SEC FAB, ITM PNIV FAB, ITM SEC M1 e ITM SEC M2)","CpfCnpj":33592510004494}]'}
{'Lista': '[{"Codigo":182,"NomeUsina":"Timbopeba","CpfCnpj":33592510041268}]'}
{'Lista': '[]'}
{'Lista': '[]'}
{'Lista': '[]'}
{'Lista': '[]'}
{'Lista': '[]'}
{'Lista': '[]'}
{'Lista': '[]'}
{'Lista': '[{"Codigo":32,"NomeUsina":"CAUÊ/CONCEIÇÃO FERRO (Usina do Cauê; Usina Conceição I e Usina Conceição II)","CpfCnpj":33592510016409}]'}
{'Lista': '[]'}
{'Lista': '[{"Codigo":51,"NomeUsina":"FÁBRICA (ITM UMD FAB, ITM SEC FAB, ITM PNIV FAB, ITM SEC M1 e ITM SEC 

KeyboardInterrupt: ignored